In [ ]:
# Setting up the Colab environment. DO NOT EDIT!
import os
import warnings
warnings.filterwarnings("ignore")

try:
    import otter, pingouin

except ImportError:
    ! pip install -q otter-grader==4.0.0, pingouin
    import otter

if not os.path.exists('walkthrough-tests'):
    zip_files = [f for f in os.listdir() if f.endswith('.zip')]
    assert len(zip_files)>0, 'Could not find any zip files!'
    assert len(zip_files)==1, 'Found multiple zip files!'
    ! unzip {zip_files[0]}

grader = otter.Notebook(colab=True,
                        tests_dir = 'walkthrough-tests')

# Walkthrough

## Learning Objectives
At the end of this learning activity you will be able to: 
 - Relate the null-hypotheses of chi2 tests, t-tests, ANOVAs, and Krustal-Wallace H-tests.
 - Calculate cross-tabulations of two categorical variables using `pd.crosstab`.
 - Employ `pg.chi2_independence` to estimate the correlation between two categorical variables.
 - Describe how to test if a distribution is normally distributed using multiple methods.
 - Employ `pg.ttest`, `pg.anova`, and `pg.kruskal` to look for differences in a dependent variable between different categorical variables.
 - Utilize post-hoc tests to refine the answer of an ANOVA.

## Introduction

As we discussed with Dr. Devlin in the introduction video, this week and next we are going to look at HIV neurocognitive impairment data from a cohort here at Drexel.
Each patient was given a full-scale neuropsychological exam and the resulting values were aggregated and normalized into Z-scores based on demographically matched healthy individuals.

In this walkthrough we will explore the demographic variables within this dataset to practice different types of hypothesis testing.
In the lab you will dive deeper and explore the effects of antiretroviral medications on neurological impairment.
In this cohort, we have two major drug regimens, d4T (Stavudine) and the newer Emtricitabine/tenofovir (Truvada).
The older Stavudine is suspected to have neurotoxic effects that are not found in the newer Truvada.
We will use inferential statistics to understand this effect.

## Hypothesis Testing

Hypothesis testing is a type of inferential statistics in which one compares a distribution to a previously assumed _null_ model that represents the _simplest_ explanation of the world.
The process of the test then involves rigorously evaluating the likelihood of observing this dataset under that _null_ assumption.
If, after rigorous evaluation, we find that the likelihood of generating this dataset under null model is below some threshold (ie p<0.05) we _reject_ the null hypothesis and accept the alternative.

While one can test arbitrarily complicated hypotheses, in biology we tend to fall back on a few go-to tests:
 * 2-factor tests (chi2-tests)
 * Tests of means (t-tests, ANOVAs, etc.)
 * Correlations (linear regression)

We'll cover the first two this week, and the last next week.

When boiled down, all of these tests rigorously balance four things:
 * Effect size : How large is _difference observed_ divided by the _spread of the observations_.
 * Type 1 Error (false-positives, p-value) : The likelihood of incorrectly _rejecting_ the null hypothesis.
 * Type 2 Error (false-negatives, power) : The likelihood of incorrectly _accepting_ the null hypothesis.
 * Number of observations.

## Pingouin

The `pingouin` Python package is a statistical tool designed to bridge the gap between the data manipulation capabilities of pandas and the visualization strengths of seaborn, with a specific focus on ease of use and accessibility.
It is particularly tailored towards researchers who are embarking on their journey into data exploration and biostatistics, leveraging their familiarity with pandas for data manipulation and seaborn for data visualization.

`pingouin` is built on top of pandas `DataFrame` objects, which means that it integrates seamlessly with the pandas ecosystem.
This allows users to effortlessly prepare their data using pandas and then apply statistical tests and models using pingouin.
The package offers a wide range of statistical functions, including t-tests, ANOVAs, correlation tests, and regression analyses, with a syntax that is more intuitive and less verbose than some of its counterparts.
This makes pingouin an attractive option for biologists who may not have a deep background in statistics but need to conduct sophisticated statistical analyses as part of their research.

Two other popular packages for statistical analysis in python are `scipy.stats` and `statsmodels`.
Scipy offers many low-level functions for calculating basic statistical tests like t-tests, ANOVAs, Mann-Whitney, etc.
However, these low level functions require extensive programming knowledge to leverage properly.

Statsmodels offers a wealth of complex statistical modeling, p-value correction tools, and allows arbitrarily complicated regression analysis.
However, due to its complexity, it is not approachable for beginners.

Under the hood, Pingouin employs `scipy.stats`, `statsmodels`, and a number of other tools into a simple and intuitive API designed to be user-friendly and accessible.
The outputs it creates are interpretable at a glance and contain outputs from a number of lower-level tools.
This is intended to make it easier to apply statistical tests and interpret their results without getting bogged down in the complexities of statistical theory.
Additionally, pingouin is designed to work well with Pandas and the Jupyter Notebook environment, which is popular among scientists for interactive data analysis.

Check out the documentation for [Pingouin](https://pingouin-stats.org/build/html/index.html).
 - [Quick Start Examples](https://pingouin-stats.org/build/html/index.html#quick-start)
 - [Available tests](https://pingouin-stats.org/build/html/api.html)
 - [Which test to pick](https://pingouin-stats.org/build/html/guidelines.html)

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import pingouin as pg

%matplotlib inline

In [ ]:
data = pd.read_csv('hiv_neuro_data.csv')
data['education'] = data['education'].astype(float)
data.head()

When studying HIV Associated Neurocognitive Disorder (HAND) we measure a number of cognitive domains through a ~90 min test battery.
The results of these tests are then aggregated into 6 cognitive domains.

- **Speed of Information Processing** : `processing_domain_z` : How quickly an individual can understand and react to the information being presented. This domain evaluates the speed at which cognitive tasks can be performed, often under time constraints.
 - **Learning and Memory** : `learningmemory_domain_z` : This involves both the acquisition of new information (learning) and the ability to retrieve this information over time (memory). It includes both verbal learning and memory (e.g., remembering words or stories) and visual learning and memory (e.g., remembering shapes or patterns).
 - **Executive Function** : `exec_domain_z` : The complex cognitive processes required for planning, organizing, problem-solving, abstract thinking, and executing strategies. This domain also encompasses decision-making and cognitive flexibility, which is the ability to switch between thinking about two different concepts or to think about multiple concepts simultaneously.
 -  **Motor Skills** : `motor_domain_z` :  This domain assesses the coordination, speed, and dexterity of motor movements, including both fine motor skills (e.g., hand movements, writing) and gross motor skills (e.g., walking, coordination).
 - **Language Skills** : `language_domain_z` : The ability to understand and produce language. While not always as prominently affected in HAND as other cognitive domains, assessments might include naming, fluency, and comprehension tasks.
 - **Sensory-Perceptual Skills** : `visuospatial_domain_z` : Although less commonly assessed in HAND research, this domain involves the ability to interpret sensory information, including visual and auditory processing.

The outcomes of these test are highly correlated with age, sex, race, and education.
This makes it difficult to compare the raw results of a test between individuals across these covariates; ie. 
is this 65 year old performing worse on a memory task relative to a 25 year old because of their _age_ or because of some type of _impairement_?
With the raw scores alone, this is impossible.

However, if we collect data from a large number of _presumably_ healthy individuals we can create a normalization scale.
These are normalized to a **Z-scale**. 
In a z-scaled value, `z = 0` is the mean; all individuals performing at the average of covariate matched controls will have a score of 0.
Each unit of a z-scale is 1-standard-deviation.
An individual with a `z = 1` is performing 1 standard-deviation _above_ thier peers while a `z = -1.5` implies that they are performing 1.5 stds _below_ their matched peers.

You've probably heard of this before: an _IQ score_ works in a similar way.
It uses a **standard scale**. This has a mean of 100 and a standard deviation of 10. 
An _average_ IQ is 100 (by definition).
An IQ of 210 (Marylin vos Savant) is 11 standard-deviations above an _average_ individual.

## Categorical comparisons

### Q1: Count the number of participants of each sex and race.

|               |    |
| --------------|----|
| Points        | 5  |
| Public Checks | 4  |

_Points:_ 5

In [ ]:
q1_race_counts = ...
q1_sex_counts = ...

In [ ]:
grader.check("q1_demographic_breakdown")

Once we have the counts, we can change each into a probability.
This represents the likelihood of observing any given instance of the categorical variable.
We can think of this like an _unfiar_ set of dice.

In [ ]:
q1_race_counts/q1_race_counts.sum()

In [ ]:
q1_sex_counts/q1_sex_counts.sum()

Let's do the same thing for the ART column.

In [ ]:
art_counts = data['ART'].value_counts()
art_counts/art_counts.sum()

I wonder if there is a relationship between demographic variables and ART therapy?

That was looking at each measurement individually, if we want to know whether they are correlated, we need a different type of table.
We need a table that measures the number of occurances of `male & Stavudine`, `female & Stavudine`, `male & Truvada`, `female & Truvada`, etc.
This is often called a _cross tabulation_.
_Traditionally_ one would use a set of `for`-loops to count this data, but Pandas has a tool for exactly this purpose.

In [ ]:
pd.crosstab(data['sex'], data['ART'], margins=True)

It can also give us frequencies.

In [ ]:
pd.crosstab(data['race'], data['ART'], margins=True, normalize='all')

## Hypothesis Testing

When testing the correlation between two _categorical_ variables we want to use a `chi2` test.
It has the _null hypothesis_ that the two categorical distributions are uncorrelated.
Rejection of that hypothesis (ie p<0.05) indicates there is some linkage between the two distributions.

Notes:

In your cross tabulation table you need _at least_ 5 observations in each cell.
Fewer than that can lead to significant losses in statistical power.

When we want to visualize a categorical comparison, it is best to use a countplot that uses both `x` and `hue` to show the two factors.
If there is a difference, we will see that the bars have inconsistent _relative_ heights when compared to each other.

In [ ]:
sns.countplot(data = data,
              x = 'sex',
              hue = 'ART')

Pingouin has a function for performing a chi2 test from a `pd.Dataframe` directly: [`pg.chi2_independence`](https://pingouin-stats.org/build/html/generated/pingouin.chi2_independence.html).
It will return three things:

In [ ]:
# Pingouin does the crosstab internally, it just wants the df and columns

expected, obs_cor, stats = pg.chi2_independence(data, 'sex', 'ART')

If these two variables were randomly associated, what would I have gotten?

In [ ]:
expected

What did I actually observe after correction?

In [ ]:
obs_cor

What is the likelihood that `obs_cor` is different from `expected`?

In [ ]:
stats

`p ~= 0.9`, therefore we cannot reject the hypothesis that ART therapy is unrelated to sex.

### Q2: Is race and education correlated in this dataset?

|               |    |
| --------------|----|
| Points        | 5  |
| Public Checks | 4  |

_Points:_ 5

In [ ]:
# Bin education into three bins:
#  * Up to HS education (0, 12]
#  * HS-4yr College (12, 16]
#  * >4yr College (16, np.inf]
# pandas has a function pd.cut that that accomplish this.

# Add this column back into the dataframe as a new column called `education_binned`
data['education_binned'] = pd.cut(data['education'], [0, 12, 16, np.inf])

In [ ]:
# Calculate the chi2 test between race and education_binned

In [ ]:
# Is race and education correlated? (yes or no)
q2_pval_ans = ...

In [ ]:
# Are there "more" or "less" H's with 12-16 years of education than we'd expect by chance
q2_inter_ans = ...

In [ ]:
grader.check("q2_demographic_education")

So, we found that there is a link between race and education, with AA and H individuals have less people in the higher education brackets and more in the lower education brackets.

In this instance we used stratification to split education into categorical variables representing meaningful cutoffs.
We can also keep the education value as a continious measurement and compare across categories.

## Continious comparisons

In order to choose the right test we need to walk down a set of decision trees to find the appropriate answer.
There are a number of places you can find these decision trees (or you can make your own), we'll use the one from the pingouin tool.

https://pingouin-stats.org/build/html/guidelines.html

The first thing to test: _normality_.

If the variable is normally distributed (or can be approximated as normal) we can use more powerful parametric tests. If it is not normal, we need to use non-parametric tests.

A `qqplot` is a common way to visualize the fit between an observed distribution and a theoretical one.
It plots the observed data against a desired distribution (usaually the normal distribution).
If there is strong agreement, it likely means the data fits the distribution.
It is the best place to start when starting considering normality.

In [ ]:
pg.qqplot(data['education'], dist='norm')

Extremely high agreement. I would expect this to be normally distributed. Let's use Pingouin to test it.

In [ ]:
# Tests the null-hypothesis: is this derived from a normal distribution
# with the default parameters

pg.normality(data['education'])

Interesting, it rejected the hypothesis that this data is normally distributed.
This is why you _NEED_ to have documentation open when utilizing these tools.

https://pingouin-stats.org/build/html/generated/pingouin.normality.html

> method: str
> Normality test.
> * ‘shapiro’ (default) performs the Shapiro-Wilk test using scipy.stats.shapiro(),
> * ‘normaltest’ performs the omnibus test of normality using scipy.stats.normaltest(),
> * ‘jarque_bera’ performs the Jarque-Bera test using scipy.stats.jarque_bera().
>
> **The Omnibus and Jarque-Bera tests are more suitable than the Shapiro test for large samples.**


In [ ]:
pg.normality(data['education'], method='normaltest')

In [ ]:
pg.normality(data['education'], method='jarque_bera')

Now we see that both of these methods, and the qq-plot agree that this data is normally disitributed.

### Two group measurement

In [ ]:
sns.barplot(data = data,
            x = 'sex',
            y = 'education')

Handy-dandy t-test.
Measures the null-hypothesis:
 * One-sample: Does this sample have a mean of _mu_?
 * Two-sample (independent): Do these two distribution have the _mean_ value?
 * Two-sample (paired): Do these pre/post measurements of the **same** sample have the same _mean_ value?

In [ ]:
male_edu = data.query('sex == "male"')['education']
female_edu = data.query('sex == "female"')['education']
pg.ttest(male_edu, female_edu)

Breaking this down:
 * `T = 2.245961` : Called the T-statistic : (mean_M-mean_F)/std_P*sqrt(1/N_m+1/N_f)
 * `dof = 202.1` : The number of samples after dropping missing and applying corrections.
 * `alternative = two-sided` : The mean of females can be larger _or_ smaller than males.
 * `p-val = 0.025` : The likelihood of seeing this difference by chance.
 * `CI95% = [0.07, 1.13]` : The difference in means between males and females is between 0.7 and 1.13 years.
 * `cohen-d = 0.26` : The effect size : (mean_M-mean_F)/std_P
 * `BF10 = 1.43` : Bayes Factor : P(D|H1)/P(D|H0) : The likelihood of the data given the alternative hypothesis divided by the likelihood of the data under the null hypothesis.
 * `power = 0.587` : The likelihood of see an effect if there truly was one given cohen-D and dof.

### Multi-group measurement

In [ ]:
sns.barplot(data = data,
            x = 'race',
            y = 'education')

Handy-dandy One-way ANOVA with post-hoc tests.

https://pingouin-stats.org/build/html/guidelines.html#anova

In [ ]:
# First test for equal variances to pick correct ANOVA

pg.homoscedasticity(data, dv='education', group='race')

In [ ]:
pg.anova(data, dv='education', between='race', detailed=True)

Here the ANOVA rejects the hypothesis that the mean is the same across races.
However, it does not tell us *which* ones are different.
That's where post-hoc testing comes in.
Since we have equal-varainces across the group, it suggets Tukey's post hoc test.

In [ ]:
pg.pairwise_tukey(data=data, dv='education', between='race')

This provided a table with a row for each comparison.
`p-tukey` indicates the rows with significant differences.

We can also do Multi-way ANOVAs; this is where you have more than one independent variable splitting your data.
Here, we can look at how both *race* **and** *gender* impact educational attainment.

In [ ]:
sns.barplot(data = data,
            x = 'race',
            hue = 'sex',
            y = 'education')

In [ ]:
pg.anova(data, dv='education', between=['race', 'sex'], detailed=True)

This tells us that there is likely a difference between race, between sex, but there likely isn't an interactive term.
We can also do the post-hoc tests across all comparisons.

In [ ]:
results = pg.pairwise_tests(data=data, dv='education', between=['race', 'sex'])
results

In [ ]:
results.loc[results['p-unc'] < 0.05]

If we had repeated measures for individuals (ie biological replicates) we can also add `, within = 'sample'`.
This would help correct for within sample variability.

### Non-parametric comparisons

### Q3: Use the appropriate non-parametric method.

Let's pretend that we believed the first Shapiro-Wilk normality test and instead used non-parametric methods.
Use the flowchart below to test the whether there is a difference in education level across `race`.

https://pingouin-stats.org/build/html/guidelines.html#non-parametric

![Non-parametric flow chart](https://pingouin-stats.org/build/html/_images/flowchart_nonparametric.svg)

|               |    |
| --------------|----|
| Points        | 5  |
| Public Checks | 4  |

_Points:_ 5

In [ ]:
# Use the appropriate test from the Guidelines
q3_stats = ...
q3_stats

In [ ]:
# Use the appropriate post-hoc test from the Guidelines
q3_post_hoc = ...
q3_post_hoc

In [ ]:
# Are they the 'same' or 'different'

q3_comparison = ...

In [ ]:
grader.check("q3_nonparametric")

We should see that we'll get the same answer between parametric and non-parametric methods.

We've now seen a handful of methods for testing the difference between populations.
In the lab, you will use these skills to test the effects of different ART drugs on neurocognitive impairment.

---------------------------------------------

## Submission

You do not need to submit this walkthrough notebook.
Simply complete the quiz.

In [ ]:
grader.check_all()